<a href="https://colab.research.google.com/github/Emmanuel-bit1234/Assignment4/blob/main/2_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Load the libraries**

In [ ]:
# !pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

import warnings
warnings.filterwarnings('ignore')

# Class Imbalance
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

# Scaling
from sklearn.preprocessing import StandardScaler

In [ ]:
file_name = "processed_dataset.csv"

data = pd.read_csv(file_name, delimiter=',', on_bad_lines='skip')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes              253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [ ]:
X = data.drop(["Diabetes"], axis=1)
y = data['Diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=42)

### **Class Imbalance**


In [ ]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
print('Before', y_train.value_counts())
print()
print('After', y_train_res.value_counts())

Before Diabetes
0.0    149592
2.0     24742
1.0      3242
Name: count, dtype: int64

After Diabetes
0.0    149592
2.0    149592
1.0    149592
Name: count, dtype: int64


### **Scaling**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

### **Multilayer Perceptron (MLP) Model**

> The MLP using Keras. The architecture can be adjusted based on the complexity of the dataset and the specific task

In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='softmax')  # Use 'softmax' if it’s a multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train_res, epochs=5)

Epoch 1/5
14025/14025 [==============================] - 49s 3ms/step - loss: -870952.0625 - accuracy: 0.3333
Epoch 2/5
14025/14025 [==============================] - 48s 3ms/step - loss: -9317841.0000 - accuracy: 0.3333
Epoch 3/5
14025/14025 [==============================] - 47s 3ms/step - loss: -34296888.0000 - accuracy: 0.3333
Epoch 4/5
14025/14025 [==============================] - 47s 3ms/step - loss: -84692464.0000 - accuracy: 0.3333
Epoch 5/5
14025/14025 [==============================] - 47s 3ms/step - loss: -169170816.0000 - accuracy: 0.3333


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

2379/2379 [==============================] - 6s 2ms/step - loss: 78109752.0000 - accuracy: 0.0183
Test Accuracy: 1.83%


### **Convolutional Neural Network (CNN)**

In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [ ]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5)

Epoch 1/5
5550/5550 [==============================] - 23s 4ms/step - loss: 0.3961 - accuracy: 0.6966
Epoch 2/5
5550/5550 [==============================] - 19s 3ms/step - loss: 0.3588 - accuracy: 0.7000
Epoch 3/5
5550/5550 [==============================] - 20s 4ms/step - loss: 0.3262 - accuracy: 0.7013
Epoch 4/5
5550/5550 [==============================] - 19s 3ms/step - loss: 0.2308 - accuracy: 0.7013
Epoch 5/5
5550/5550 [==============================] - 20s 4ms/step - loss: -0.0242 - accuracy: 0.7027


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

2379/2379 [==============================] - 5s 2ms/step - loss: -0.3406 - accuracy: 0.7055
Test Accuracy: 70.55%


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Display the column names
print(data.columns)

# Replace 'Diabetes' with the column you want to predict
target_column = 'Diabetes'

# Features and target
X = data.drop(columns=[target_column])
y = data[target_column]

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode the target labels if they are categorical
if data[target_column].dtype == 'object':
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Reshape the features for RNN input (samples, time steps, features)
# Here we consider each feature as a time step
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

Index(['Diabetes', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')
Epoch 1/10
5074/5074 [==============================] - 88s 17ms/step - loss: 0.7300 - accuracy: 0.7323 - val_loss: 0.4817 - val_accuracy: 0.7479
Epoch 2/10
5074/5074 [==============================] - 84s 17ms/step - loss: 0.4055 - accuracy: 0.7169 - val_loss: 0.3985 - val_accuracy: 0.6477
Epoch 3/10
5074/5074 [==============================] - 83s 16ms/step - loss: 0.4143 - accuracy: 0.7136 - val_loss: 0.3583 - val_accuracy: 0.7426
Epoch 4/10
5074/5074 [==============================] - 86s 17ms/step - loss: 0.4545 - accuracy: 0.7253 - val_loss: 0.4067 - val_accuracy: 0.7325
Epoch 5/10
5074/5074 [==============================] - 84s 17ms/step - loss: 0.4289 -

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset
file_name = "processed_dataset.csv"
data = pd.read_csv(file_name, delimiter=',', on_bad_lines='skip')

# Display the column names
print(data.columns)

# Replace 'Diabetes' with the column you want to predict
target_column = 'Diabetes'

# Features and target
X = data.drop(columns=[target_column])
y = data[target_column]

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode the target labels if they are categorical
if data[target_column].dtype == 'object':
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Reshape the features for LSTM input (samples, time steps, features)
# Here we consider each feature as a time step
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


Index(['Diabetes', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')
Epoch 1/10
5074/5074 [==============================] - 67s 13ms/step - loss: 0.5835 - accuracy: 0.7184 - val_loss: 0.4208 - val_accuracy: 0.6861
Epoch 2/10
4068/5074 [=======================>......] - ETA: 13s - loss: 0.4097 - accuracy: 0.7180